In [1]:
import os, glob, oracledb
from sqlalchemy import create_engine
os.chdir(r'C:\Users\silmao\OneDrive - Dufry International AG\Documentos\Gamma\Fechamento mensal\2405')
# Path TNSNAMES
oracledb.init_oracle_client(config_dir='C:\Oracle\instantclient_21_10\network\admin')

In [ ]:
!pip install -q cx_Oracle

In [2]:
pysqldf = lambda q: sqldf(q, globals())

In [3]:
import pandas as pd
import datetime as dt
import pandasql
import numpy as np
from datetime import datetime
from pandasql import sqldf

In [4]:
# Conectando base Oracle
def conecta_oracle(usu, senha, sid):
    #conn = oracledb.connect(user = usu, password = senha, dsn = sid)
    engine = create_engine('oracle+cx_oracle://'+usu+':'+senha+'@'+sid)
    return engine

In [5]:
lista_arquivos = [f for f in os.listdir('.') if f.startswith("INPEREBF") & f.endswith(".xlsx")]

In [6]:
lista_arquivos

['INPEREBF_BD_2405.xlsx', 'INPEREBF_BR_2405.xlsx']

In [7]:
if 'base' in locals():
    del(base)
    del(saldo_aerop)

In [8]:
for f in lista_arquivos:
    temp = pd.read_excel(f)
    temp = temp[(temp["Saldo disponible calculado"]).notnull() & (temp["Fec.Inic Permanencia"]).notnull()]
    temp = temp[temp["Saldo disponible calculado"] > 0]
    temp = temp.drop([c for c in temp.columns if c.startswith("Unnamed")], axis=1)
    colunas_saldo = temp.columns[28:len(temp.columns)-1]
    colunas_saldo = colunas_saldo.insert(0, temp.columns[12])
    saldo_temp = temp.loc[:, colunas_saldo]
    temp = temp.iloc[:, [0,1,3,6,8,9,10,11,12,15,24,25]]
    if 'base' in locals():
        base = pd.concat([base, temp], ignore_index=True)
        saldo_aerop = saldo_aerop.merge(saldo_temp, on = "Articulo", how = "outer")
    else:
        base = temp
        saldo_aerop = saldo_temp

In [9]:
# Filtrando pelos depositos de BD e BR, caso existam outros não desejados
base = base[base["Deposito"].isin(["DA27197888", "DA17625216"])]

In [10]:
aeroportos = saldo_aerop.columns[1:len(saldo_aerop.columns)]

In [11]:
aeroportos

Index(['BSB', 'CNF', 'FOR', 'GIG', 'NAT', 'REC', 'FLN', 'GRU', 'POA', 'VCP',
       'SSA'],
      dtype='object')

In [12]:
saldo_aerop = saldo_aerop.fillna(0)
saldo_aerop = saldo_aerop.drop_duplicates()
saldo_aerop = saldo_aerop.rename(columns={"Articulo": "item_code"})
saldo_aerop["item_code"] = saldo_aerop["item_code"].astype(str)

In [ ]:
saldo_aerop.head(10)

In [13]:
saldo_aerop = saldo_aerop.groupby(['item_code']).max()

In [14]:
saldo_aerop = saldo_aerop.melt(var_name = "airport", value_name = "Saldo", ignore_index=False).reset_index()

In [15]:
# datas de referencia 
dt_base = base.loc[0, "Fecha proceso"].replace(day=1)
mes_ref = (dt_base - dt.timedelta(days=1)).strftime("%Y%m")

In [16]:
dt_inicio_mes = (dt_base - dt.timedelta(days=1)).strftime("01-%m-%Y")
dt_fim_mes = (dt_base - dt.timedelta(days=1)).strftime("%d-%m-%Y")

In [17]:
# Abre conexão com base Oracle RJD27 para recuperar data primeiro recebimento anterior a 1 ano
db = conecta_oracle("gecex", "u89dw037b1", "rjd27")

In [18]:
item_recente = pd.read_sql_query("select cod_produto item_code, min(nvl(p.dat_desembaraco, dat_di)) primeira_entrada " +
        "from exp_invoice_it it, exp_invoice i, exp_procimp p " +
        "where it.cod_empresa = i.cod_empresa " +
        "and it.cod_filial = i.cod_filial " + 
        "and it.cod_documento = i.cod_documento "+
        "and p.cod_empresa = i.cod_empresa " +
        "and p.cod_filial = i.cod_filial " +
        "and p.num_procimp = i.num_procimp " +
        "having min(nvl(p.dat_desembaraco, dat_di)) > add_months(to_date(''' "+ dt_base.strftime("01-%m-%Y") +" ''', '''dd-mm-yyyy'''), -12) " + 
        "group by cod_produto" , db)

In [19]:
# Encerrando conexão com banco de dados
db.dispose()

In [20]:
item_recente["meses"] = ((dt_base - item_recente["primeira_entrada"]).dt.days/30).astype(int)
item_recente = item_recente.drop(["primeira_entrada"], axis = 1)
item_recente.loc[item_recente["meses"]==0, "meses"] = 1

In [ ]:
item_recente.head(10)

In [21]:
# Abre conexão com base Oracle RJD28HST para recuperar venda por aeroporto
db = conecta_oracle("CMD2022", "CmD2o24", "rjd28hst")

In [22]:
vnda_aerop = pd.read_sql_query("select substr(airport_code,4,3) airport, item_code, sum(units) sales "+
"from sales_by_ticket_new " +
"where real_sales_date between add_months(to_date('''"+ dt_base.strftime("01-%m-%Y") +"''', '''dd-mm-yyyy'''), -12) and to_date('''"+ dt_fim_mes +"''', '''dd-mm-yyyy''') " +
"and substr(airport_code,1,2) in ('BR', 'BD') "+
"group by substr(airport_code,4,3) , item_code", db)

In [23]:
# Encerrando conexão com banco de dados
db.dispose()

In [24]:
vnda_aerop = vnda_aerop.merge(item_recente, on = "item_code", how = 'left')
vnda_aerop["meses"] = vnda_aerop["meses"].fillna(12)
vnda_aerop.loc[vnda_aerop["sales"] < 0, "sales"] = 0
vnda_aerop["sales"] = round(vnda_aerop["sales"]/vnda_aerop["meses"],2)
vnda_aerop = vnda_aerop.rename(columns={"sales": "VM"})

In [ ]:
vnda_aerop.head(10)

In [25]:
saldo_aerop = saldo_aerop.merge(vnda_aerop, on = ["item_code", "airport"], how = 'left')
saldo_aerop["VM"] = saldo_aerop["VM"].fillna(0)
saldo_aerop = saldo_aerop.drop(["meses"], axis = 1)
saldo_aerop["Perm"] = round(saldo_aerop["Saldo"] / saldo_aerop["VM"], 2)
saldo_aerop["Risco"] = 0

In [ ]:
saldo_aerop.head(10)

In [26]:
saldo_aerop = saldo_aerop.pivot(index = "item_code", columns= ["airport"], values=["Saldo", "VM", "Perm", "Risco"]).reset_index()
saldo_aerop = saldo_aerop.fillna(0)

In [27]:
det = ["Saldo_", "VM_", "Perm_", "Risco_"]
colunas = ["item_code"]
for d in det:
    for a in aeroportos.sort_values():
        colunas.append(d+a)

In [ ]:
colunas

In [28]:
# Renomeadndo colunas do data Frame
saldo_aerop.columns = colunas

In [29]:
colunas = ["item_code"]
for a in aeroportos:
    for d in det:
        colunas.append(d+a)

In [30]:
# Reordenando colunas do data Frame
saldo_aerop = saldo_aerop[colunas]

In [31]:
saldo_aerop = saldo_aerop.rename(columns={"item_code": "Artigo"})

In [ ]:
saldo_aerop.head()

In [32]:
del(vnda_aerop)

In [33]:
base.isnull().any()

Fecha proceso                 False
Deposito                      False
Fec.Inic Permanencia          False
Fec. Impor.                    True
Días Stock                    False
Tester                        False
Categoría                     False
Marca                          True
Articulo                      False
Descripción                   False
Saldo disponible calculado    False
Valor Total disponible        False
dtype: bool

In [34]:
base = base.loc[base["Fec.Inic Permanencia"].notnull()]

In [35]:
base = base.loc[base["Valor Total disponible"].notnull()]

In [36]:
base["Marca"] = base["Marca"].fillna("N/A")

In [37]:
base["Empresa"] = ["DLF" if i == 'DA17625216' else "DDB" for i in base["Deposito"]]

In [38]:
base = base.drop(['Fecha proceso', 'Deposito'], axis = 1)

In [39]:
base["Classificacao"] = ["FOR SALE" if i == "N" else "TTG" for i in base["Tester"]]

In [40]:
base["mes_inicio"] = base["Fec.Inic Permanencia"].dt.strftime("%Y%m")

In [41]:
base["Descripción"] = base["Descripción"].str.upper()

In [42]:
base = base.drop(base.columns[0:4], axis = 1)

In [ ]:
base.head(10)

In [43]:
aging = base.groupby(by=["Empresa","mes_inicio","Categoría","Marca","Articulo","Descripción","Classificacao"], as_index = False).sum()

In [ ]:
aging.head()

In [44]:
aging.columns = ["Empresa", "Mes_inicio", "Categoria", "Marca", "Artigo","Descricao", "Classificacao", "Saldo_Vencer", "Valor_Vencer"]

In [45]:
aging["Artigo"]=aging["Artigo"].astype(str)

In [46]:
# buscando venda últimos 12 meses
vm = pd.read_csv(str.replace("L12_BD&BR_Sales&DamageQty_Aging_MESREF.csv", "MESREF", mes_ref), sep = ";", encoding='latin1', usecols=[0,1,7,8])

In [ ]:
vm.head(10)

In [47]:
vm = vm.merge(item_recente, left_on = "LOCAL_ITEM_CODE", right_on = "item_code", how = 'left')
vm["meses"] = vm["meses"].fillna(12)
vm = vm.drop(["item_code"], axis = 1)
vm.loc[vm["SALES_QTY"] < 0 , "SALES_QTY"] = 0

In [48]:
vm["VMedia Mensal"] = round(vm["SALES_QTY"] /vm["meses"], 2)
vm["DESMedia Mensal"] = round(vm["DAMAGE_QTY"] / vm["meses"], 2)
vm["BU_CODE"] = ["DDB" if i == 'BD' else "DLF" for i in vm["BU_CODE"]]

In [49]:
vm = vm.drop(["SALES_QTY", "DAMAGE_QTY", "meses"], axis=1)
vm.columns= ["Empresa", "Artigo", "VMedia Mensal","DMedia Mensal"]

In [50]:
del(item_recente)

In [51]:
# montando df final
aging= aging.merge(vm, on=["Empresa", "Artigo"], how='left')

In [ ]:
aging.head(10)

In [52]:
aging["VMedia Mensal"] = aging["VMedia Mensal"].fillna(0)
aging["DMedia Mensal"] = aging["DMedia Mensal"].fillna(0)

In [53]:
# Setando TTG como Tester ou GWP
gwp = ["BRINDE", "AMOSTRA", "GWP", "BRD ", "GIFT", "BOLSAS TAG EUROPEAS"]
aging.loc[aging["Classificacao"]=="TTG", "Classificacao"] = ["GWP" if any(w in c for w in gwp) else "TTG" for c in aging[aging["Classificacao"]=="TTG"]["Descricao"] ]
aging.loc[aging["Classificacao"]=="TTG", "Classificacao"] = ["GWP" if m > 0 else "TTG" for m in aging[aging["Classificacao"]=="TTG"]["VMedia Mensal"]]
aging["Classificacao"] = ["TESTER" if c == "TTG" else c for c in aging["Classificacao"]]


In [54]:
aging["Classificacao"].unique()

array(['FOR SALE', 'TESTER', 'GWP'], dtype=object)

In [55]:
aging.loc[aging["Classificacao"]=="TESTER", "VMedia Mensal"] = aging[aging["Classificacao"]=="TESTER"]["DMedia Mensal"]
aging = aging.drop("DMedia Mensal", axis = 1)

In [56]:
aging["mes_vencimento"] = aging["Mes_inicio"].astype(int) + 500

In [57]:
aging["dt_mes_vencimento"] = pd.to_datetime(aging["mes_vencimento"], format="%Y%m")

In [58]:
aging["meses_para_vencer"] = ((aging["dt_mes_vencimento"] - dt_base).dt.days/30).astype(int)

In [59]:
aging["consumo_acumulado"] = (aging["meses_para_vencer"] * aging["VMedia Mensal"]).astype(int)

In [60]:
aging["saldo_acumulado"] = aging.groupby(["Empresa","Artigo"])["Saldo_Vencer"].cumsum()

In [61]:
aging["perda_vencimento"] = aging["saldo_acumulado"] - aging["consumo_acumulado"]

In [62]:
aging.sort_values(by=['Empresa','Artigo','mes_vencimento'], inplace=True)

In [63]:
aging.reset_index(inplace = True)
aging = aging.drop(['index'], axis = 1)

In [64]:
# calculando saldo acumulado menos perda do periodo anterior
aging["saldo_acumulado_menos_perda"] = aging["perda_vencimento"].shift()
aging.loc[(aging["Empresa"]+aging["Artigo"] != aging["Empresa"].shift()+aging["Artigo"].shift()) , "saldo_acumulado_menos_perda"] = 0 
aging.loc[aging["saldo_acumulado_menos_perda"] < 0, "saldo_acumulado_menos_perda"] = 0 
aging["saldo_acumulado_menos_perda"] = aging["saldo_acumulado"] - aging["saldo_acumulado_menos_perda"]

In [65]:
aging["perda_vencimento"] = aging["saldo_acumulado_menos_perda"] - aging["consumo_acumulado"]

In [66]:
aging.loc[aging["perda_vencimento"] < 0, "perda_vencimento"] = 0

In [67]:
aging["risco"] = ["SIM" if p > 0 else "NAO" for p in aging["perda_vencimento"] ]

In [68]:
aging["vl_risco"] = aging["perda_vencimento"] * (aging["Valor_Vencer"]/aging["Saldo_Vencer"])

In [69]:
# computando Totais
aging_total = aging.groupby(by=["Empresa","Classificacao","mes_vencimento"], as_index = False)["vl_risco"].sum()
aging_total.columns = ["Empresa", "Classificacao", "Vencimento", "Valor Risco"]

In [ ]:
aging_total

In [70]:
aging_total.sort_values(by = ['Classificacao', 'Empresa', 'Vencimento'], inplace = True, ignore_index = True)

In [71]:
aging_total["Valor Risco"] = aging_total["Valor Risco"].astype(int)

In [72]:
aging_total = aging_total.pivot_table(index="Vencimento", columns= ["Classificacao", "Empresa"], values="Valor Risco", fill_value = 0).reset_index()

In [73]:
# Totalizadores por classificação
qtde_classificacoes = len(aging["Classificacao"].unique())
for c in range(1, qtde_classificacoes+1):
    aging_total['TOT '+ aging_total.columns[c+(c-1)][0]] = aging_total.iloc[:, c+(c-1) : c+(c-1)+2].sum(axis = 1)

In [74]:
aging_total['TOTAL'] = aging_total.iloc[:, (qtde_classificacoes*2)+1 : (qtde_classificacoes*3)+2].sum(axis = 1)

In [75]:
aging_total.columns[1:len(aging_total)+1]

MultiIndex([(    'FOR SALE', 'DDB'),
            (    'FOR SALE', 'DLF'),
            (         'GWP', 'DDB'),
            (         'GWP', 'DLF'),
            (      'TESTER', 'DDB'),
            (      'TESTER', 'DLF'),
            ('TOT FOR SALE',    ''),
            (     'TOT GWP',    ''),
            (  'TOT TESTER',    ''),
            (       'TOTAL',    '')],
           names=['Classificacao', 'Empresa'])

In [ ]:
aging_total.columns[1][0]

In [76]:
# Totalizadores por categoria
total_cat = aging.groupby(by=["Categoria","mes_vencimento"], as_index = False)["vl_risco"].sum()
total_cat.columns = ["Categoria", "Vencimento", "Valor Risco"]
total_cat["Valor Risco"] = total_cat["Valor Risco"].astype(int)

In [77]:
total_cat = total_cat.pivot_table(index="Vencimento", columns= ["Categoria"], values="Valor Risco", fill_value = 0).reset_index()

In [78]:
total_cat["TOTAL"] = total_cat.iloc[:, 1:len(total_cat.axes[1])+1].sum(axis = 1)

In [ ]:
total_cat

In [79]:
# Acrescentando saldo por aeroporto
aging = aging.merge(saldo_aerop, on=["Artigo"], how='left')
aging.sort_values(by = ["Empresa", "Artigo"], inplace = True, ignore_index = True)
aging = aging.drop(["dt_mes_vencimento"], axis = 1)

In [ ]:
# se aeroporto_principal
#   risco = menor(saldo_aerop, saldo_acumulado) - (vm_aerop * meses_vencer)

# senao
#   se vm_principal * meses_vencer < saldo_acumulado ou saldo_principal < saldo_acumulado entao -- principal nao vai conseguir vender todo saldo a vencer, tem que calcular risco saldo filial
#      risco = menor(saldo_filial, saldo_acumulado-(menor(vm_princial*meses_vencer, saldo_principal))) - (vm_filial * meses_vencer)

# se risco < 0 entao risco = 0

In [80]:
# Calculo Risco aeroportos
aeroportos_ddb = ["BSB","CNF","FOR","GIG","NAT","REC"]
principal_ddb = aeroportos.get_loc("GIG")
principal_dlf = aeroportos.get_loc("GRU")
aerop_indx = 0

In [81]:
def condicoes(r):
    if r["Empresa"] == "DDB":
        if ((((r[aging.columns[((principal_ddb-1)*4)+19]] * r["meses_para_vencer"]) < r["saldo_acumulado"]) | (r[aging.columns[((principal_ddb-1)*4)+18]] < r["saldo_acumulado"])) & (aeroportos[aerop_indx] in aeroportos_ddb)):
            return min(r[aging.columns[((i-1)*4)+18]], (r["saldo_acumulado"]-min((r[aging.columns[((principal_ddb-1)*4)+19]] * r["meses_para_vencer"]) , r[aging.columns[((principal_ddb-1)*4)+18]]))) - (r[aging.columns[((aerop_indx-1)*4)+19]] * r["meses_para_vencer"])
        else: 
            return 0
    elif r["Empresa"] == "DLF":   
        if ((((r[aging.columns[((principal_dlf-1)*4)+19]] * r["meses_para_vencer"]) < r["saldo_acumulado"]) | (r[aging.columns[((principal_dlf-1)*4)+18]] < r["saldo_acumulado"])) &  ( aeroportos[aerop_indx] not in aeroportos_ddb)):
            return min(r[aging.columns[((i-1)*4)+18]], (r["saldo_acumulado"]-min((r[aging.columns[((principal_dlf-1)*4)+19]] * r["meses_para_vencer"]) , r[aging.columns[((principal_dlf-1)*4)+18]]))) - (r[aging.columns[((aerop_indx-1)*4)+19]] * r["meses_para_vencer"])
        else: 
            return 0

In [82]:
for i in range(1, aeroportos.size):
    aerop_indx = i
    if ((i == principal_ddb) | (i == principal_dlf)):
        aging[aging.columns[i*4+17]] = np.where(((aging["Empresa"]=='DDB') & (aerop_indx == principal_ddb) ) | ((aging["Empresa"]=='DLF') & (aerop_indx == principal_dlf) ), np.where(aging[aging.columns[((i-1)*4)+18]] < aging["saldo_acumulado"], aging[aging.columns[((i-1)*4)+18]], aging["saldo_acumulado"] ) - (aging[aging.columns[((i-1)*4)+19]] * aging["meses_para_vencer"]),aging[aging.columns[i*4+17]])
    else:
        # se aerop principal não vai conseguir vender todo saldo a vencer, então calcula risco saldo filial
        aging[aging.columns[i*4+17]] = aging.apply(condicoes, axis=1)
    aging.loc[aging[aging.columns[i*4+17]] < 0, aging.columns[i*4+17]] = 0

In [83]:
with pd.ExcelWriter('Aging_DF_MESREF.xlsx'.replace('MESREF', mes_ref)) as writer:
    aging.to_excel(writer, sheet_name = 'Detail')
    aging_total.to_excel(writer, sheet_name = 'Total Risk')
    total_cat.to_excel(writer, sheet_name = 'Total Category')

In [ ]:
#FIM

In [ ]:
for i in range(1, aeroportos.size):
    aging[aging.columns[i*4+17]] = 0

In [ ]:
aging.loc[aging["Empresa"]=="DDB", "Risco_FLN"].sum()

In [ ]:
aging.columns

In [ ]:
aging_total.to_excel('aging_py.xlsx')

In [ ]:
aging.to_excel('aging_pyton.xlsx')

In [ ]:
aging.loc[aging["Artigo"]=="2012734"]

In [ ]:
aeroportos.get_loc("GIG")

In [ ]:
str(aging["mes_vencimento"])

In [ ]:
aging.apply(lambda row : row.shift(), axis = 0)

In [ ]:
saldo_aerop.loc[saldo_aerop["airport"] == 'BSB']["Saldo"].sum()

In [ ]:
aging["Saldo_Vencer"].sum()

In [ ]:
aging_total["TOTAL"].sum()

In [ ]:
aging["vl_risco"].sum()

In [ ]:
total_cat["TOTAL"].sum()

In [ ]:
type(aeroportos_ddb)

In [ ]:
aging[aging.columns[i*4+17]] = np.where(((aging["Empresa"]=='DDB') & (aeroportos[i] == principal_ddb) ) | ((aging["Empresa"]=='DLF') & (aeroportos[i] == principal_dlf) ), np.where(aging[aging.columns[((i-1)*4)+18]] < aging["saldo_acumulado"], aging[aging.columns[((i-1)*4)+18]], aging["saldo_acumulado"] ) - (aging[aging.columns[((i-1)*4)+19]] * aging["meses_para_vencer"]),aging[aging.columns[i*4+17]])

In [ ]:
principal_ddb

In [ ]:
base["Valor Total disponible"].sum()

In [ ]:
base.columns

In [ ]:
base.loc[base["Deposito"] != 'DA17625216']["Saldo disponible calculado"].sum()

In [ ]:
base.loc[base["Articulo"] == 1009480]

In [ ]:
base.loc[base["Deposito"] != 'DA17625216']

In [ ]:
teste = pd.read_excel('INPEREBF_BD_2310.xlsx')

In [ ]:
teste["Valor Total disponible"].sum()

In [ ]:
teste = teste[(teste["Saldo disponible calculado"]).notnull() & teste["Saldo disponible calculado"] > 0 & (teste["Fec.Inic Permanencia"]).notnull()]

In [ ]:
testeb = teste[(teste["Saldo disponible calculado"]).notnull() & (teste["Fec.Inic Permanencia"]).notnull()]

In [ ]:
testeb["Valor Total disponible"].sum()

In [ ]:
testeb = testeb[testeb["Saldo disponible calculado"] > 0]

In [ ]:
base.loc[base["Deposito"] == 'FR17625216']

In [ ]:
base["Deposito"].unique()